### Problem Statement ###

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.o reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

**Churn Phases**
- In ‘good’ phase the customer is happy with the service and behaves as usual
- In ‘action’ phase The customer experience starts to sore in this phase
- In ‘churn’ phase the customer is said to have churned

#### Business Goal ####

In this project, you will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

#### Outcomes ####

- Predict churn only on high-value customers
- Predict usage-based definition to define churn


### Step 1: Data Exploration

In [ ]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Importing Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:.2f}'.format)


plt.style.use('fivethirtyeight')

In [ ]:
telecom_df = pd.read_csv('telecom_churn_data.csv')

In [ ]:
# Let's see the head of our master dataset
telecom_df.head()

In [ ]:
# Let's check the dimensions of the dataframe
telecom_df.shape

In [ ]:
# let's look at the statistical aspects of the dataframe
telecom_df.describe(include='all')

In [ ]:
# Let's see the type of each column
telecom_df.info(verbose=True)

In [ ]:
## Below columns are breaking the convention. So we will rename them appropriately
vbc_cols = [col for col in telecom_df.columns if 'vbc' in col]
print(vbc_cols)

In [ ]:
telecom_df.rename(columns = {'jun_vbc_3g': 'vbc_3g_6', 'jul_vbc_3g': 'vbc_3g_7', 'aug_vbc_3g': 'vbc_3g_8', 'sep_vbc_3g': 'vbc_3g_9'}, inplace=True)

In [ ]:
#DataType Correction 
object_df = telecom_df.select_dtypes(include='object')
object_df.head()

#Looks like all are datetime object

In [ ]:
#convert object to date time

for col in object_df.columns:
    telecom_df[col] = pd.to_datetime(telecom_df[col])

In [ ]:
telecom_df.shape

We are now ready to cleanse the data and create a Manageable Data Set for further processing

## Step 2 : Data Cleansing

**Common Utility Functions**

In [ ]:
# Function which returns the columns with missing values > the cutoff percentage
# Argument: cutoff percentage between 1 - 100
def calculate_missing_values(data, cutoff):
    missing_percent= round(data.isna().sum() / len(data.index) * 100)
    print("{} features having more than {}% missing values:".format(len(missing_percent[missing_percent > cutoff]), cutoff))
    return missing_percent[missing_percent > cutoff]

#Function to handle missing values across months 
# Argument: Take list of column names without month number suffix 
def impute_zero_in_missing(data, columnList):
    for feature in [col + suffix for suffix in ['_6','_7','_8','_9'] for col in columnList]:
        data[feature].fillna(0, inplace=True)
        

#Function to drop columns across months
def drop_columns(data,columnList):
    for feature in [col+suffix for suffix in ['_6','_7','_8','_9'] for col in columnList]:
        data.drop([feature],inplace=True, axis=1)

Drop columns which have only null values

In [ ]:
telecom_df[telecom_df.isnull().all(axis=1)]

##Looks like there are no columns with only null values

Drop all columns with 1 unique value 

In [ ]:
unique_cols = telecom_df.nunique()
unique_cols[unique_cols == 1]

In [ ]:
telecom_df.drop(unique_cols[unique_cols == 1].index, inplace=True, axis=1)

In [ ]:
telecom_df.shape

Drop columns with missing values greater than 70% or impute the same

In [ ]:
calculate_missing_values(telecom_df, 50)

Impute Missing Value as 0 for all the **Recharge, Revenue, Night Pack, Fb User** Fields

In [ ]:
missingValueColumnList=['total_rech_data', 'max_rech_data', 'count_rech_2g', 'count_rech_3g', 'av_rech_amt_data'
                       , 'arpu_3g', 'arpu_2g', 'night_pck_user', 'fb_user']

In [ ]:
# Since the minimum value is 1, we are going to handle NA values by imputing with 0 
# which means we are assuming there were no recharges done by the customer
impute_zero_in_missing(telecom_df, missingValueColumnList)

In [ ]:
calculate_missing_values(telecom_df, 50)

In [ ]:
calculate_missing_values(telecom_df, 7)

We will drop all features which have more than 70% missing values

In [ ]:
drop_columns(telecom_df,['date_of_last_rech_data'])

In [ ]:
calculate_missing_values(telecom_df, 7)

All columns except date columns are for the month 9 i.e the Churn phase. 
This data will be eventually dropped when we tag churn/no churn, 
hence we will be skipping imputing these fields related to month 9

### Step 3: Data Preparation ###

**Filter in High Value Customers which are the target of our analysis**

* A high-value customers is defined as follows:

- Those who have recharged with an amount more than or equal to X, where X is greater than 70th percentile of the average recharge amount in the first two months (the good phase)

In [ ]:
# Create new column total recharge amount for data for aiding in finding High Value customer 

telecom_df['total_rech_amt_data_6'] = telecom_df.av_rech_amt_data_6 * telecom_df.total_rech_data_6
telecom_df['total_rech_amt_data_7'] = telecom_df.av_rech_amt_data_7 * telecom_df.total_rech_data_7
telecom_df['total_rech_amt_data_8'] = telecom_df.av_rech_amt_data_8 * telecom_df.total_rech_data_8

In [ ]:
#Create column for holding average of total recharge amount for good phase (the months June (6) and July (7))
# We add the total recharge amount of call and data and find the average across two months
telecom_df['total_avg_rech_amnt_Good_Phase'] = (telecom_df.total_rech_amt_6 + telecom_df.total_rech_amt_data_6 \
                                               + telecom_df.total_rech_amt_7+ telecom_df.total_rech_amt_data_7)/2

In [ ]:
# filter values greater than 70th percentile of total average recharge amount for good phase 
seventieth_percentile = telecom_df.total_avg_rech_amnt_Good_Phase.quantile(0.7)

telecom_df_high_val_cust = telecom_df[telecom_df.total_avg_rech_amnt_Good_Phase > seventieth_percentile]

In [ ]:
print("70th Percentile of Average Recharge amount in Good Phase (June and July month) is ", seventieth_percentile)

In [ ]:
telecom_df_high_val_cust.shape


**Define the Target Variable Churn based on the following criteria**

* A Churned Customer is defined as follows : 
* A Customer has churned (churn=1, else 0) if in the Ninth Month he/she has not made any calls (either incoming or outgoing) 
* AND have not used mobile internet even once. 



In [ ]:
#Add a new column "churn", values would be either 1 (churn) or 0 (non-churn)
telecom_df_high_val_cust['churn'] = \
        np.where(telecom_df_high_val_cust[['total_ic_mou_9','total_og_mou_9', \
                                           'vol_2g_mb_9', 'vol_3g_mb_9']].sum(axis=1) == 0, 1,0)

In [ ]:
# Find out the % of churn/non churn customers
churn_percentage = telecom_df_high_val_cust.churn.value_counts(normalize=True)
print(churn_percentage)
churn_percentage.plot.bar()
plt.show()

**Observation** : The churn percentage is around 8%. This indicates that there is a slight imbalance in the dataset which will need to be corrected in modelling

**Drop all data of the ninth Month as that is our Target Variable**


In [ ]:
churn_month_columns =  telecom_df_high_val_cust.columns[telecom_df_high_val_cust.columns.str.contains('_9')]

In [ ]:
churn_month_columns

In [ ]:
# drop all columns corresponding to the churn phase
telecom_df_high_val_cust.drop(churn_month_columns,axis=1,inplace=True)

In [ ]:
telecom_df_high_val_cust.shape

### Step 4 : EDA ### 

We will now perform EDA and try to get insights into the data. 
Based on the insights we could define our approach to training, remove outliers, remove highly correlated variables

In [ ]:
#CheckPoint 1 
telecom_eda_df = telecom_df_high_val_cust.copy()


#### 4.1 Bi variate Analysis of Various Variables with the Churn Variable ####
We will draw trends of the selected categorical variables data wrt to the Label, Churn and see if we can derive any meaningful insights

**Utility Functions**

In [ ]:
# create box plot for  6th, 7th and 8th month
def create_box_plot(column):
    plt.figure(figsize=(15,10))
    df = telecom_eda_df
    plt.subplot(2,3,1)
    sns.boxplot(data=df, y=column+"_6",x="churn", showfliers=False)
    plt.subplot(2,3,2)
    sns.boxplot(data=df, y=column+"_7",x="churn", showfliers=False)
    plt.subplot(2,3,3)
    sns.boxplot(data=df, y=column+"_8",x="churn", showfliers=False)
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
    
    


In [ ]:
# create box plot for  6th, 7th and 8th month
def create_bar_plot(column):
    plt.figure(figsize=(15,10))
    df = telecom_eda_df
    plt.subplot(2,3,1)
    sns.barplot(data=df, y=column+"_6",x="churn")
    plt.subplot(2,3,2)
    sns.barplot(data=df, y=column+"_7",x="churn")
    plt.subplot(2,3,3)
    sns.barplot(data=df, y=column+"_8",x="churn")
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
def showbarlabel(graph, rotate=0):
    graph.set_xticklabels(graph.get_xticklabels(),rotation=rotate)
    for p in graph.patches:
        height = p.get_height()
        graph.text(p.get_x()+p.get_width()/2., height + 0.1,height ,ha="center")

In [ ]:
def plot_bar(by,rotate=0):
    df = telecom_df_high_val_cust
    graph = df[by].value_counts(sort=True).plot(kind='bar')
    showbarlabel(graph,rotate)

In [ ]:
# Ploting for Total recharge amount :
create_box_plot('total_rech_amt')

**Observation** : Total Recharge amount drops in month 8 indicating Churn 

In [ ]:
# Ploting for maximum recharge amount :
create_box_plot('max_rech_amt')

**Observation**  : We can see that there is a huge drop in maximum recharge amount for churned customers in the 8th month i.e action phase


In [ ]:
create_box_plot('av_rech_amt_data')

**Observation** : Average Recharge amount drops in month 8 indicating Churn 

In [ ]:
# Ploting for total recharge amount data :
create_box_plot('total_rech_amt_data')


**Observation** : We can see a drop in the total recharge for data for churned customers in the 8th Month i.e Action Phase

In [ ]:
# all Recharge Number related column list
recharge_num_columns = [col for col in telecom_df_high_val_cust if 'rech_num' in col.lower()]
recharge_num_columns

In [ ]:
100*telecom_df_high_val_cust[recharge_num_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
# Ploting for total recharge number:
create_box_plot('total_rech_num')

**Observation** : We can see that there is a huge drop in total recharge number for churned customers in the 8th month i.e action phase

In [ ]:
# all Recharge data related column list
recharge_data_columns = [col for col in telecom_df_high_val_cust if 'rech_data' in col.lower()]
recharge_data_columns

In [ ]:
100*telecom_df_high_val_cust[recharge_data_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
# Ploting for total recharge data:
create_box_plot('total_rech_data')

**Observation** : Again we can see that there is a huge drop in total recharge amount data for churned customers in the 8th month i.e action phase

In [ ]:
# Ploting for max recharge for data:
create_box_plot('max_rech_data')

**Observation** : There is a huge drop in max recharge amount data for churned customers in the 8th month i.e action phase 

In [ ]:
# Ploting for Last  day recharge amount  :
create_box_plot('last_day_rch_amt')

In [ ]:
# Ploting for volume of 2G and 3G usage columns:
create_box_plot('vol_2g_mb')
create_box_plot('vol_3g_mb')

**Observation** 
We see 2g and 3g usage for churned customers drops in the 8th month i.e Action phase.

However in general we see the usage is low for churned customer across months. 

In [ ]:
# Ploting for count of 2G and 3G recharge columns:
create_bar_plot('count_rech_2g')
create_bar_plot('count_rech_3g')

**Observation** 
We see 2g and 3g recharge counts for churned customers drops in the 8th month i.e Action phase.



In [ ]:
# Ploting for arpu of 2G and 3G usage columns:
create_box_plot('arpu_2g')
create_box_plot('arpu_3g')

In [ ]:
# Ploting bar plot for arpu of 2G and 3G usage columns:
create_bar_plot('arpu_2g')
create_bar_plot('arpu_3g')

**Observation** 
We see 2g and 3g arpu for churned customers drops in the 8th month i.e Action phase.



In [ ]:
# Ploting for monthly subcription of 2G and 3G usage columns:
create_box_plot('monthly_2g')
create_box_plot('monthly_3g')

In [ ]:
# Plotting a bar plot as box plot doesn't show any pattern
create_bar_plot('monthly_2g')
create_bar_plot('monthly_3g')

**Observation** 
We see 2g and 3g monthly subscription for churned customers drops in the 8th month i.e Action phase.



In [ ]:
# Plotting for small duration subscription of 2g and 3g data
# Plotting a bar plot as box plot doesn't show any pattern
create_bar_plot('sachet_2g')
create_bar_plot('sachet_3g')

**Observation** 
We see 2g and 3g small duration subscription for churned customers drops in the 8th month i.e Action phase.





In [ ]:
#Plotting volume based 3g usage
create_bar_plot('vbc_3g')

In [ ]:
#Getting the  day  columns
day_columns = [col for col in telecom_df_high_val_cust if 'day' in col.lower()]
day_columns

In [ ]:
100*telecom_df_high_val_cust[day_columns].isnull().sum()/len(telecom_df_high_val_cust.index)
# We don't have any missing values here

In [ ]:
create_box_plot('last_day_rch_amt')

Huge drop in 8th month for last day recharge amount indicating  churn

In [ ]:
# all Date column list
date_columns = [col for col in telecom_df_high_val_cust if 'date' in col.lower()]
date_columns

In [ ]:
100*telecom_df_high_val_cust[date_columns].isnull().sum()/len(telecom_df_high_val_cust.index)


The missing value indicates that recharge date and the recharge value are missing together which means the customer didn't recharge for that month

In [ ]:
telecom_df_high_val_cust[telecom_df_high_val_cust.date_of_last_rech_6.isnull()][['total_rech_data_6','date_of_last_rech_6']].head()

In [ ]:
#Plot for ARPU 
create_box_plot('arpu')

**Observation** 
We see the ARPU for churned customers drops in the 8th month i.e Action phase.



In [ ]:
#Plot for Onnet Minutes of Usage
create_box_plot('onnet_mou')

The calls on service provider network drops in month 8 indicates Churn

In [ ]:
#Plot for Offnet Minutes of Usage
create_box_plot('offnet_mou')

The calls to different network drops in month 8 indicates Churn

#### General Observation from above EDA #### 
- We see churned customers drops in the 8th month i.e Action phase.
- We can also see that the trend reversal happens drastically in the 9th month. 
- In the 6th and 7th month, the variance captured between the Churned and Non Churned customers is not much.
- So to reduce dimensionality it might be a good idea to average out the values of both the columns and drop the individual months. 
- We can test this hypothesis in the models later


#### 4.2 Dimensionality Reduction  ####

Based  on the EDA done above, we will reduce the dimensionality to make forecasting of the trends easier based on the  features



**Dimensionality Reduction | Drop Highly Correlated Columns as a PreRequisite to EDA**

In [ ]:
## Creating a copy to avoid regenerating master for each iterations
##Check point 2
training_df = telecom_eda_df.copy()

In [ ]:
#Find Highly correlated data and drop Highly Correlated Columns
cor = training_df.corr()
cor.loc[:,:] = np.tril(cor, k=-1)
sns.heatmap(cor, cmap='Greens', annot=False, )
plt.show()

Looks like there are strong multicollinearity issues, Lets drop data which is multi collinear

In [ ]:
# Create correlation matrix
corr_matrix = training_df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.85
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
to_drop

In [ ]:
## Filter columns of months where only all months are highly correlated, others we will ignore 
to_drop_filtered = ['std_og_t2t_mou_6',
 'std_og_t2t_mou_7',
 'std_og_t2t_mou_8',
 #'std_og_t2m_mou_7',
 #'std_og_t2m_mou_8',
# 'isd_og_mou_7',
# 'isd_og_mou_8',
 #'total_og_mou_8',
 'total_ic_mou_6',
 'total_ic_mou_7',
 'total_ic_mou_8',
 'total_rech_amt_6',
 'total_rech_amt_7',
 'total_rech_amt_8',
 'count_rech_2g_6',
 'count_rech_2g_7',
 'count_rech_2g_8',
 'arpu_2g_6',
 'arpu_2g_7',
 'arpu_2g_8',
 'sachet_2g_6',
 'sachet_2g_7',
 'sachet_2g_8',
# 'monthly_3g_7',
#'monthly_3g_8',
 'sachet_3g_6',
 'sachet_3g_7',
 'sachet_3g_8',
]

training_df.drop(to_drop_filtered, axis =1, inplace=True)


**Dimensionality Reduction | Drop the Date Columns as they are already factored in the other columns, Drop Mobile Number as its not meaningful to prediction**

In [ ]:
training_df.drop(['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','mobile_number'], axis=1, inplace=True)

In [ ]:
training_df.shape

In [ ]:
#Find Highly correlated data and drop Highly Correlated Columns
cor = training_df.corr()
cor.loc[:,:] = np.tril(cor, k=-1)
sns.heatmap(cor, cmap='Greens', annot=False)
plt.show()

In [ ]:
#Checkpoint 3
# We can run from checkpoints rather than run the entire notebook for validation
telecom_df_high_val_cust = training_df.copy()

**Dimensionality Reduction | Avg Out the 6th and 7th Month to reduce the number of Features**

In [ ]:
# Creating Avg Column 

col_list = telecom_df_high_val_cust.filter(regex='_6|_7').columns.str[:-2]
col_list.unique()

print (telecom_df_high_val_cust.shape)

for idx, col in enumerate(col_list.unique()):
    print(col)
    avg_col_name = "avg_"+col+"_av67"
    col_6 = col+"_6"
    col_7 = col+"_7"
    telecom_df_high_val_cust[avg_col_name] = (telecom_df_high_val_cust[col_6]  + telecom_df_high_val_cust[col_7])/ 2

In [ ]:
#Drop the individual columns

print (telecom_df_high_val_cust.shape)

col_list = telecom_df_high_val_cust.filter(regex='_6|_7').columns

telecom_df_high_val_cust.drop(col_list, axis=1, inplace=True)
telecom_df_high_val_cust.shape

#### Univariate Analysis 

**Analyse sample of  categorical variables and see if we can draw any meaningful insights**

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    counter = 0;
    for i in range(min(nCol,nGraphShown)):
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), pd._libs.tslibs.timestamps.Timestamp)):
            plt.subplot(nGraphRow, nGraphPerRow, counter+1)
            #increment the counter
            counter +=1
            if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
                valueCounts = columnDf.value_counts()
                valueCounts.plot.bar()
            else:
                columnDf.hist()
                plt.ylabel('counts')
                plt.xticks(rotation = 90)
                plt.title(f'{columnNames[i]} (column {i})')
        
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
plotPerColumnDistribution(telecom_df_high_val_cust,30,8)

**Observation** -
* Histogram distribution shows twin peaks(bimodal) or single peaks on the lower bars of the distribution.
* The distribution is not normal, which indicates this could be a higher order linear or non linear distribution. 
* Smoothening may be required for better prediction. 
* The distribution is sparse or spread in one bucket

**Analysis of sample of  Numeric/Continuous Features to see if we can derive any trends**

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
plotScatterMatrix(telecom_df_high_val_cust, 25, 10)

**Observation** :
* Relation between the variables are not obviously linear
* All numeric variables are having distributions which are not normal


**Analysis of Tenure with Churn**

In [ ]:
#Convert AON in Months
telecom_df_high_val_cust['aon_mon'] = telecom_df_high_val_cust['aon']/30
telecom_df_high_val_cust.drop('aon', axis=1, inplace=True)
telecom_df_high_val_cust['aon_mon'].head()

In [ ]:
telecom_df_high_val_cust.aon_mon.describe()

In [ ]:
ax = sns.distplot(telecom_df_high_val_cust['aon_mon'], hist=True, kde=False, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax.set_ylabel('No of Customers')
ax.set_xlabel('Tenure (months)')
ax.set_title('Customers by their tenure')

In [ ]:
tn_range = [0, 6, 12, 24, 60, 61]
tn_label = [ '0-6 Months', '6-12 Months', '1-2 Yrs', '2-5 Yrs', '5 Yrs and above']
telecom_df_high_val_cust['tenure_range'] = pd.cut(telecom_df_high_val_cust['aon_mon'], tn_range, labels=tn_label)
telecom_df_high_val_cust['tenure_range'].head()

In [ ]:
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

temp = pd.Series(data = 'tenure_range')
fig, ax = plt.subplots()
width = len(telecom_df_high_val_cust['tenure_range'].unique()) + 6 + 4*len(temp.unique())
fig.set_size_inches(width , 7)

total = float(len(telecom_df_high_val_cust.index))
ax = sns.countplot(x="tenure_range", data=telecom_df_high_val_cust, palette="Set2", hue = "churn");
for p in ax.patches:
                ax.annotate('{:1.1f}%'.format((p.get_height()*100)/float(len(telecom_df_high_val_cust))), (p.get_x()+0.05, p.get_height()+20))
plt.xticks(rotation=90)
plt.show()


**Correlation of  Churn with all Variables**

In [ ]:
#Get Correlation of "Churn" with other variables:
plt.figure(figsize=(20,10))
telecom_df_high_val_cust.corr()['churn'].sort_values(ascending = False).plot(kind='bar')
plt.show()

**Observation**

*Positive Corelation with*
- Churn has got maximum correlation with "Avg Std Outgoing Month 6 and 7"
- Roaming Outgoing/Incoming Minutes

*Negative Corelation with*
- fb Usage Plan for 8th Month
- arpu 8th Month
- Total Outgoing 8th Month

Lets also validate if ARPU8 and TotalRech8 are correlated, and if so we should eliminate one of these variables

**Relation between ARPU8 and TotalRecharge Num in 8th Month**

In [ ]:
telecom_df_high_val_cust[['total_rech_num_8', 'arpu_8']].plot.scatter(x = 'total_rech_num_8',
                                                              y='arpu_8')
plt.show()

**Observation** : Looks like they are not correlated and we will keep both the variables

## Step 3: Model Building

Moving towards Model Building
Also drop a **Mobile Number** and **Tenure**


### 3.1 Pre Training Steps ###

In [ ]:
## Checkpoint 5

model_df = telecom_df_high_val_cust[:].copy()
telecom_df_high_val_cust.shape


In [ ]:
#Dropping tenure_range since we have AON MONTH already and columns are highly coorelated
model_df.drop('tenure_range', axis=1, inplace=True)

#dropping total_avg_rech_amnt_Good_Phase which was calculated to find High Value customers.

model_df.drop('total_avg_rech_amnt_Good_Phase', axis=1, inplace=True)

#Since All The Values are realted to Price/ Cost/ Amount, Filling NaN with 0

model_df.fillna(0, inplace=True)

model_df.head()


In [ ]:
# Creating response and predictor variables
response='churn'
predictor=model_df.columns[model_df.columns != 'churn']


#### Split into Train and Test Sets

In cases when classification problems  can exhibit a large imbalance in the distribution of the target classes,  it is recommended to use stratified sampling to ensure that relative class frequencies is approximately preserved in each train and validation fold.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(model_df, test_size=0.3, random_state=0,stratify=model_df[response])

#Rows and columns after split
print (df_train.shape)
print (df_test.shape)

Scale all the predictor variables, so that there is equal weightage to all the features, and convergence is faster

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_train[predictor] = scaler.fit_transform(df_train[predictor])
df_test[predictor] = scaler.transform(df_test[predictor])


In [ ]:
df_train.churn.value_counts()

In [ ]:
# X_train, y_train
X_train = df_train.drop('churn', axis=1)
y_train = df_train['churn']

# X_test, y_test
X_test = df_test.drop('churn', axis=1)
y_test = df_test['churn']

In [ ]:
#Applying SMOTE
#We wont be applying Smote on the Test Data Set

import imblearn
from imblearn.over_sampling import SMOTE
try:
    print(imblearn.__version__)
except:
    print("Please install SMOTE Package First")


In [ ]:
sampling=SMOTE(random_state = 0)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

def perform_pca_plot_scree_plot(X):
    pca = PCA(random_state=0)
    pca.fit(X)
    fig = plt.figure(figsize = (8,6))
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative  variance explained')
    plt.show()
    return pca

In [ ]:
pca = perform_pca_plot_scree_plot(df_train[predictor])

**Analysis:**

Looks like 60 components are enough to describe 95% of the variance in the dataset.We'll choose 60 components for our modeling

In [ ]:
col = list(df_train[predictor].columns)
df_pca = pd.DataFrame({'PC1':pca.components_[0],'PC2':pca.components_[1], 'PC3':pca.components_[2],'Feature':col})
df_pca.head(10)

### 3.2 Model Training ###

*Approach*
- We will first train a Logistical Regression Model, which will form our Baseline
- We will then try  to try a Kernel and Random Forest and try to improve on our baseline
- Kernel Model we will use PCA and optimise for Prediction 
- Random Forest Model we will optimise Forecasting and use it for find the important features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score, make_scorer
from sklearn.svm import SVC
import itertools

In [ ]:
# This function plots the confusion matrix.
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
## For y_true and y_pred display the Classification Metrics and plot Confusion Matrix
def computeClassificationMetrics(y_test,y_test_pred, plot=True):
        print('*'*20+"Classification Report"+'*'*20)
        print(classification_report(y_test,y_test_pred))
        
        if plot:
            # Compute confusion matrix
            cnf_matrix = confusion_matrix(y_test, y_test_pred)
            print('*'*20+'Confusion Matrix'+'*'*20)
            # Plot non-normalized confusion matrix
            class_names = [0,1]
            plt.figure(figsize=(7,5))
            plot_confusion_matrix(cnf_matrix
                      , classes=class_names)                      
            plt.show()
      
        

In [ ]:
##Function for training models and testing on a test set. if plot is True, it plots a PRC curve for 
##training and test sets and finds the threshold where (precision*recall) is maximum.
def model_fit(alg, X_train_data, y_train_data, X_test_data, y_test_data, plot=True):
    
    #Fit the algorithm on the data
    alg.fit(X_train_data, y_train_data)
        
    #Predict training set:
    y_train_pred = alg.predict(X_train_data)
    y_test_pred = alg.predict(X_test_data)
      
    computeClassificationMetrics(y_test, y_test_pred, plot)
    

In [ ]:
## For CV reusults plot the  C Value vs Accuracy Results        
def display_svm_stats(cv_results,param_value):
    gamma = cv_results[cv_results['param_gamma']==param_value]
    plt.plot(gamma["param_C"], gamma["mean_test_score"])
    plt.plot(gamma["param_C"], gamma["mean_train_score"])
    plt.xlabel('C')
    plt.ylabel('Accuracy')
    plt.title("Gamma="+str(param_value))
    plt.ylim([0.60, 1])
    plt.legend(['test accuracy', 'train accuracy'], loc='lower right')
    plt.xscale('log') 


#### Balancing by over-sampling minority class using SMOTE ####

We will start by balancing the data set using SMOTE technique. First we will balance the data set by SMOTE and then apply logistic regression model on the balanced data set.
https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

- SMOTE technique should be applied only on the training data, after splitting the data set into training set and validation set. 
- If we apply SMOTE before splitting the data, we would leak the information in validation set into the training set. 





In [ ]:
#Imbalance before SMOTE
print("X_train Shape : ", X_train.shape)
print("X_test Shape : ", X_test.shape)

y_train_imb = (y_train != 0).sum()/(y_train == 0).sum()
y_test_imb = (y_test != 0).sum()/(y_test == 0).sum()
print("Imbalance in Train Data : ", y_train_imb)
print("Imbalance in Test Data : ", y_test_imb)

In [ ]:
print("Shape of train dataset before SMOTE ", df_train[predictor].shape)
x_tr,y_tr = sampling.fit_sample(df_train[predictor],df_train[response])
print("Shape of train dataset after SMOTE : ", x_tr.shape)

# Applying PCA : Principal Component Analysis using 60 components
pca = IncrementalPCA(n_components=60)    
X_train_pca = pca.fit_transform(x_tr)
X_test_pca = pca.transform(df_test[predictor])
print("Shape of train dataset after PCA : ", X_train_pca.shape)

In [ ]:
print("X_tr Shape", x_tr.shape)
print("y_tr Shape", y_tr.shape)

imb = (y_tr != 0).sum()/(y_tr == 0).sum()
print("Imbalance in Train Data : ",imb)


### Logistic Regression 


We will training a logistic regression and test it on a validation set. 


#### Baseline Logistic Regression Model

In [ ]:
# Basic Logistric Regression
logreg1 = LogisticRegression(random_state = 0)


In [ ]:
# Call model fit and evalute the validation set
model_fit(logreg1,X_train_pca, y_tr, X_test_pca, df_test[response] ,True)

We acheived a Accuracy of 83% an F1 Score of 0.86. This is a very decent prediction power on a unbalanced data set using Logistic Regression. 

#### Model Iteration 1: Using SVM with Appropriate Kernel for Predictive Model ####

- Choose the right kernel 

In [ ]:
#rbf Kernel

lr = LogisticRegression(random_state=0)
lr.svm = SVC(kernel='rbf') 

# Call model fit and evalute the validation set
model_fit(lr.svm ,x_tr, y_tr, df_test[predictor],df_test[response] ,True)



In [ ]:
# Poly Kernel
lr = LogisticRegression(random_state=0)

lr.svm = SVC(kernel='poly') 
model_fit(lr.svm ,x_tr, y_tr, df_test[predictor],df_test[response] ,True)


In [ ]:
#Linear Kernel
lr = LogisticRegression(random_state=0)

lr.svm = SVC(kernel='linear') 
model_fit(lr.svm ,x_tr, y_tr, df_test[predictor],df_test[response] ,True)


**Analysis** : 
Since the Test set is unbalanced we need to look at the weighted score, for precision and Recall. 
Looks like the **Poly** Kernel is giving the 89% accuracy score with an F1 value of 0.91. This is really good. 
This also is in line with our understanding from EDA. 

First we will try to combine this with PCA and see if we can capture the variance in lower dimensions

In [ ]:
# Applying PCA to Kernel

lr = LogisticRegression(random_state=0)

lr.svm = SVC(kernel='poly') 
model_fit(lr.svm,X_train_pca, y_tr, X_test_pca, df_test[response] ,True)


**Analysis** : Its very clear that PCA degrades the SVM Score. 
- The reason could be that PCA projects the values in the linear space and that is causing some information to be lost, in such a way that the SVM Kernel cannot read it.
- https://www.researchgate.net/post/Is_there_a_specific_reason_that_using_PCA_gives_worse_results_than_without_using_it_in_SVM_classification
- We will NOT be using PCA for SVM. 
- Lets next try to optimise the C and gamma hyperparameters for the SVM and see if we can improve the results

In [ ]:
# # commenting as grid search is taking a long time on local workstation
# # creating a KFold object with 5 splits 
# folds = KFold(n_splits = 3, shuffle = True, random_state = 101)

# # Set the parameters by cross-validation
# hyper_params = [ {'gamma': [1e-1,1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}]
# #hyper_params = [ {'gamma': [1e-1], 'C': [1]}]

# # specify model
# model = SVC(kernel="poly")

# # set up GridSearchCV()
# model_cv_svm = RandomizedSearchCV(estimator = model, 
#                         param_distributions = hyper_params, 
#                         scoring= 'accuracy', 
#                         cv = folds,
#                         n_jobs = -1,
#                         verbose = 2,
#                         return_train_score=True, random_state=100)      

# # fit the model
# model_cv_svm.fit(X_tr, y_tr)  


# ## This will take some time, and you should twiddle your thumbs

In [ ]:
# Commenting as grid search is taking a long time
# # cv results
# svm_cv_results = pd.DataFrame(model_cv_svm.cv_results_)
# svm_cv_results['param_C'] = svm_cv_results['param_C'].astype('int')
# gamma=[1e-1,1e-2, 1e-3, 1e-4]
# plt.figure(figsize=(16,5))
# plt.subplot(141)
# display_svm_stats(svm_cv_results,gamma[0])
# plt.subplot(142)
# display_svm_stats(svm_cv_results,gamma[1])
# plt.subplot(143)
# display_svm_stats(svm_cv_results,gamma[2])
# plt.subplot(144)
# display_svm_stats(svm_cv_results,gamma[3])
# plt.show()

**Analysis**

We wil be using the default values, as GridSearch is a brute force algorithm, and is taking long time to process on desktop workstation. 
Also with the poly kernel we are getting 89% Accuracy with Fscore of 0.91 which is better than our baseline

In [ ]:
# # printing the optimal accuracy score and hyperparameters
# best_score = model_cv_svm.best_score_
# best_hyperparams = model_cv_svm.best_params_

# log("The best test score is {0} corresponding to hyperparameters {1}".format(round(best_score,2), best_hyperparams))

**Analysis**

We wil be using the default values, as GridSearch is a brute force algorithm, and is taking long time to process on desktop workstation. 
Also with the poly kernel we are getting 89% Accurity with Fscore of 90 which is better than our baseline

In [ ]:
#Store session so that heavy computation results can be persisted in memory
#import dill
#dill.dump_session('notebook_env.db')

 #### Model Iteration 2: Using RandomForest 

In [ ]:
# Importing random forest classifier from sklearn library
from sklearn.ensemble import RandomForestClassifier


In [ ]:
rfc = RandomForestClassifier(random_state=0)

In [ ]:
# Call model fit and evalute the validation set
model_fit(rfc,X_train_pca, y_tr, X_test_pca, df_test[response] ,True)

The baseline Random Forest regression model gave a Recall of 92% and Precision of 92% on the validation set.

### Hyperparameter Tuning using GridSearchCV



In [ ]:
def tune_hyperparameter(parameters,x_train,y_train,n_folds = 5,max_depth=0):
    
    if(max_depth==0):
        rf = RandomForestClassifier(random_state=0)
    else :
        rf = RandomForestClassifier(max_depth=max_depth, random_state=0)
        
    rf = GridSearchCV(rf, parameters, cv=n_folds,n_jobs = -1, scoring="accuracy",return_train_score=True)
    rf.fit(x_train, y_train)
    scores = rf.cv_results_

    for key in parameters.keys():
        hyperparameters = key
        break

    # plotting accuracies for parameters
    plt.figure(figsize=(16,5))
    plt.plot(scores["param_"+hyperparameters], scores["mean_train_score"], label="training accuracy")
    plt.plot(scores["param_"+hyperparameters], scores["mean_test_score"], label="test accuracy")
    plt.xlabel(hyperparameters)
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

**Tuning max_depth**

Let's try to find the optimum values for max_depth and understand how the value of max_depth impacts the overall accuracy of the ensemble.

In [ ]:
# parameters to build the model on
parameters = {'max_depth': range(2, 30, 5)}
tune_hyperparameter(parameters,X_train_pca, y_tr)

**Analysis:**

We can see that as we increase the value of max_depth, both train and test scores increase till a point, but after that test score becomes stagnant.
12 and 18 value have peek convergence and can be used for grid veiw search.

In [ ]:
# parameters to build the model on
parameters = {'n_estimators': range(100, 1000, 200)}
tune_hyperparameter(parameters,X_train_pca, y_tr)

**Analysis:** 
Score almost remain the same with very low dip throught the range. We can use 200 for grid view search.

**Tuning max_features**

Let's find how the model performance varies with max_features, which is the maximum number of features considered for splitting at a node.

In [ ]:
# parameters to build the model on
parameters = {'max_features': [20,30,40]}
tune_hyperparameter(parameters,X_train_pca, y_tr,4)

**Analysis**

Apparently, accuracy of training seems to be stable and test scores seems to decrease after 30 

In [ ]:
# parameters to build the model on
parameters = {'min_samples_leaf': range(1, 50, 10)}
tune_hyperparameter(parameters,X_train_pca, y_tr)

**Analysis:** 

10 to 20 seems to be a good range and that will be used in grid search.

In [ ]:
# parameters to build the model on
parameters = {'min_samples_split': range(10, 50, 10)}
tune_hyperparameter(parameters,X_train_pca, y_tr)

**Analysis:** 

Range 10 to 20 is optimal with good accuracy.

In [ ]:
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [12,18],
    'n_estimators': [200],
    'max_features': [30],
    'min_samples_leaf': [10,20],
    'min_samples_split': [10,20]
}

rf = RandomForestClassifier(random_state=0)
# Instantiate the grid search model
rf_grid_search = GridSearchCV(estimator = rf, param_grid = params, 
                          cv = 5, n_jobs = -1,verbose = 1, scoring= "accuracy", 
                          return_train_score=True)

In [ ]:
rf_grid_search.fit(X_train_pca, y_tr)

In [ ]:
print("We can get accuracy of {} using \n{}".format(round(rf_grid_search.best_score_,2),rf_grid_search.best_params_))

Building and Evaluating the Final Model for Random Forest


In [ ]:
rfc = RandomForestClassifier(max_depth=18,
                             max_features=30,
                             min_samples_leaf=10,
                             min_samples_split=20,
                             n_estimators=200,
                             n_jobs = -1,random_state=0)
#Call model fit with validate set

model_fit(rfc,X_train_pca, y_tr, X_test_pca, df_test[response] ,True)

#### Model Iteration 3: Using XGB for Forecasting Model with Feature Importance ####

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

In [ ]:
# fit model on training data with default hyperparameters
model = XGBClassifier(random_state=0)
model_fit(model,x_tr,y_tr, df_test[predictor],df_test[response] ,True)

In [ ]:
# hyperparameter tuning with XGBoost

# creating a KFold object 
folds = 5

# specify range of hyperparameters
param_grid = {'learning_rate': [0.1,0.2,0.3], 
             'subsample': [0.3,0.4,0.5]}          


# specify model
xgb_model = XGBClassifier(random_state=0)

#auc scorer
scorer = make_scorer(roc_auc_score,
                             greater_is_better=True,
                             needs_proba=True,
                             needs_threshold=False)

# set up GridSearchCV()
model_cv = GridSearchCV(estimator = xgb_model, 
                        param_grid = param_grid, 
                        scoring= "accuracy", # auc score
                        cv = folds, 
                        n_jobs = -1,
                        verbose = 1,
                        return_train_score=True)      

In [ ]:
# fit the model
model_cv.fit(x_tr,y_tr)

In [ ]:
# cv results
cv_results_xboost = pd.DataFrame(model_cv.cv_results_)
cv_results_xboost['param_learning_rate'] = cv_results_xboost['param_learning_rate'].astype('float')

In [ ]:
# printing the optimal accuracy score and hyperparameters
print('We can get accuracy score of **'+str(round(model_cv.best_score_,2))+'** using '+str(model_cv.best_params_))

In [ ]:
def plot_for_xboost(param_grid,cv_results):
    plt.figure(figsize=(18,5))
    for n, subsample in enumerate(param_grid['subsample']):
        # subplot 1/n
        plt.subplot(1,len(param_grid['subsample']), n+1)
        df = cv_results[cv_results['param_subsample']==subsample]

        plt.plot(df["param_learning_rate"], df["mean_test_score"])
        plt.plot(df["param_learning_rate"], df["mean_train_score"])
        plt.xlabel('learning_rate')
        plt.ylabel('Accuracy')
        plt.title("subsample={0}".format(subsample))
        plt.ylim([0.60, 1])
        plt.legend(['test score', 'train score'], loc='lower right')
        plt.xscale('log')

In [ ]:
param_grid1 = {'learning_rate': [0.1,0.2,0.3], 'subsample': [0.3,0.4,0.5]}  
plot_for_xboost(param_grid1,cv_results_xboost)

#### Building and Evaluating the Final Model ####

In [ ]:
#chosen hyperparameters
# fit model on training data

model = XGBClassifier(learning_rate = 0.2, subsample = 0.5, random_state=0)
model_fit(model,x_tr,y_tr, df_test[predictor],df_test[response] ,True)


#### Feature Importance ####

In [ ]:
# plot
plt.bar(range(len(model.feature_importances_)), model.feature_importances_)
plt.show()

**Observation** - Some features are more important than the others. So definetely  they can contribute towards forecasting

In [ ]:
# plot feature importance

col = list(X_train.columns)
model.get_booster().feature_names = col

ax =plot_importance(model.get_booster(),max_num_features=15)
figure = ax.figure
figure.set_size_inches=(30,45)

**Analysis**

The most important feature is **The total num of times recharge is done for the 8th Month**. This is by far the most important feature

The other important features are **Last day recharge Amt** for the 8th month, **Total Recharge Data** for 8th month, **Max Recharge Amt** for the 8th month 

So in summary, the Recharge value and Frequency of the 8th Month is a significant indicator of churn. This is logical and makes sense.

## Step 4 : Model Training Summary ##

#### Training Summary ####

#### Business Recommendations ####